In [132]:
import findspark
findspark.init()

from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import*
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd
import json

# For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.


In [128]:
def read_data():
    file=input("Which file would you like to extract? Note: Please include file extension")
    while ".json" not in file:
        print("Please make sure this is a json file and include the file extension. Enter again: ")
        file=input("Which file would you like to extract? Note: Please include file extension")
    spark = SparkSession.builder.master("local[*]").appName(file).getOrCreate()
    df = spark.read.json(file)
    print("Extraction Completed")
    if "branch" in file:
        df = df.na.fill(value=99999, subset=["BRANCH_ZIP"])
        global branch_df
        branch_df =df
    elif "custmer" in file:
        df = df.withColumn("FIRST_NAME", f.initcap(df.FIRST_NAME))
        df = df.withColumn("MIDDLE_NAME", f.lower(df.MIDDLE_NAME))
        df = df.withColumn("LAST_NAME", f.initcap(df.LAST_NAME))
        df = df.withColumn("STREET_NAME,APT_NO", f.concat(f.col("STREET_NAME"), f.lit(","), f.col("APT_NO")))
        df = df.drop("APT_NO", "STREET_NAME")
        global customer_df
        customer_df = df
    elif "credit" in file:
        df = df.withColumn("DAY,MONTH,YEAR", f.concat(f.col("YEAR"), f.col("MONTH"), f.col("DAY")))
        df = df.drop("DAY", "MONTH", "YEAR")
        global credit_df
        credit_df = df
    print("Transformation Complete")

read_data()


23/03/01 11:47:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
Extraction Completed
Transformation Complete


In [133]:
def write_data():
    start_write=input("Would you like to load this data into MySQL DB? (y/n)")
    if start_write in ["yes", "y"]:
        print("Loading...")
        branch_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH").option("user", "root").option("password", "password").save()
        customer_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER").option("user", "root").option("password", "password").save()
        credit_df.write.format("jdbc").mode("append").option("url", "jdbc:mysql://localhost:3306/creditcard_capstone").option("dbtable", "creditcard_capstone.CDW_SAPP_CREDIT_CARD").option("user", "root").option("password", "password").save()
    print("Finished Writing And Loading Data to Database")

write_data()


Loading...


Py4JJavaError: An error occurred while calling o711.save.
: java.sql.SQLSyntaxErrorException: Unknown database 'creditcard_capstone'
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:120)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:833)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:453)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:246)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:122)
	at org.apache.spark.sql.